# Preprocess essential climate variables

This notebook preprocesses data on sea surface salinity, chlorophyll-a concentration and land surface temperature. Before, the Bash and Python scripts `1_download_the_data/download_sea_surface_salinity_data.sh`, `1_download_the_data/download_chlorophyll_a_concentration_data.sh` and `1_download_the_data/download_and_preprocess_land_surface_temperature_data.py` need to be executed to download the respective data.

The Bash scripts `1_download_the_data/download_sea_surface_salinity_data.sh` and `1_download_the_data/download_chlorophyll_a_concentration_data.sh` download all the respective data from 2010 to 2018 at once. `1_download_the_data/download_and_preprocess_land_surface_temperature_data.py` downloads data only for a single month at a time and needs to be executed accordingly to download all the available data from 2010 to 2018 for all months.

## Setup

In [1]:
# import packages
import os
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
from pyproj import CRS
import gc

In [2]:
# make list of desired years
years = list(np.arange(2010, 2019))

In [3]:
# set min and max values for longitude and latitude
lon_min, lon_max, lat_min, lat_max = 60, 100, 0, 40

In [4]:
def preprocess_ecv(file, ecv, mask_lon, mask_lat):
    
    """
    This function preprocesses a given essential climate variable (ecv).
    It first loads a given file into an xarray.
    It then selects the relevant dimensions and ecv and crops the xarray to the relevant bounding box based on given longitude and latitude masks.
    It then extracts year and month from time and drops time.
    Finally, it converts the xarray into a dataframe and drops rows where data on the given ecv is missing.
    """
    
    all_data = xr.open_dataset(file)
    preprocessed_data = all_data[['time', 'lat', 'lon', ecv]].where(mask_lon & mask_lat, drop=True).to_dataframe().dropna(subset=[ecv]).reset_index()
    preprocessed_data['year'] = preprocessed_data['time'].apply(lambda x: x.year)
    preprocessed_data['month'] = preprocessed_data['time'].apply(lambda x: x.month)
    preprocessed_data = preprocessed_data.drop('time', axis=1)
    
    return preprocessed_data

## Sea surface salinity

First, a sample is explored before preprocessing all the data and saving it.

In [5]:
path = '../../data/sea_surface_salinity'

In [6]:
file = os.path.join(path, str(years[0]), sorted(os.listdir(os.path.join(path, str(years[0]))))[0])

In [7]:
data = xr.open_dataset(file)

In [8]:
data

<xarray.Dataset>
Dimensions:           (lat: 584, lon: 1388, time: 1)
Coordinates:
  * lat               (lat) float32 -83.52 -81.98 -80.7 ... 80.7 81.98 83.52
  * lon               (lon) float32 -179.9 -179.6 -179.4 ... 179.4 179.6 179.9
  * time              (time) datetime64[ns] 2010-01-15
Data variables:
    sss               (time, lat, lon) float32 ...
    sss_random_error  (time, lat, lon) float32 ...
    noutliers         (time, lat, lon) float32 ...
    total_nobs        (time, lat, lon) float32 ...
    pct_var           (time, lat, lon) float32 ...
    sss_qc            (time, lat, lon) int8 ...
    lsc_qc            (time, lat, lon) int8 ...
    isc_qc            (time, lat, lon) int8 ...
Attributes: (12/42)
    creation_time:             27-Apr-2021 00:36:28
    title:                     ESA CCI Sea Surface Salinity ECV produced at a...
    institution:               ACRI-ST,LOCEAN
    Conventions:               CF-1.8
    summary:                   ESA CCI Sea Surface Salinity
    keywords:                  Ocean, Ocean Salinity, Sea Surface Salinity, S...
    ...                        ...
    product_version:           3.21
    format_version:            CCI Data Standards v2.3
    comment:                   The following caveats must be considered: 1) P...
    creator_url:               https://climate.esa.int/fr/projects/sea-surfac...
    id:                        ESACCI-SEASURFACESALINITY-L4-SSS-MERGED_OI_Mon...
    history:

In [9]:
data.info()

xarray.Dataset {
dimensions:
	lat = 584 ;
	lon = 1388 ;
	time = 1 ;

variables:
	float32 lat(lat) ;
		lat:long_name = latitude ;
		lat:units = degrees_north ;
		lat:standard_name = latitude ;
		lat:valid_min = -90.0 ;
		lat:valid_max = 90.0 ;
	float32 lon(lon) ;
		lon:long_name = longitude ;
		lon:units = degrees_east ;
		lon:standard_name = longitude ;
		lon:valid_min = -180.0 ;
		lon:valid_max = 180.0 ;
	datetime64[ns] time(time) ;
		time:long_name = time ;
		time:standard_name = time ;
	float32 sss(time, lat, lon) ;
		sss:standard_name = sea_surface_salinity ;
		sss:valid_min = 0.0 ;
		sss:valid_max = 50.0 ;
		sss:ancillary_variables = noutliers total_nobs sss_qc ;
		sss:long_name = Multi-Satellite Sea Surface Salinity ;
	float32 sss_random_error(time, lat, lon) ;
		sss_random_error:long_name = Sea Surface Salinity Random Error ;
		sss_random_error:valid_min = 0.0 ;
		sss_random_error:valid_max = 100.0 ;
		sss_random_error:ancillary_variables = pct_var ;
	float32 noutliers(time, lat

In [10]:
# create masks to reduce the data to the desired longitudes and latitudes
mask_lon = (data['lon'] >= lon_min) & (data['lon'] <= lon_max)
mask_lat = (data['lat'] >= lat_min) & (data['lat'] <= lat_max)

In [11]:
%%time

# loop over the desired years, load each file, preprocess it, append it to a geodataframe and save it
for year in years:
    
    print(f'Processing {year}...')
    
    files = os.listdir(os.path.join(path, str(year)))
    sss_preprocessed = pd.DataFrame(columns=['lat', 'lon', 'sss', 'year', 'month'])
    
    # load files and apply preprocessing function
    for file in files:
        sss_preprocessed_temp = preprocess_ecv(os.path.join(path, str(year), file), 'sss', mask_lon, mask_lat)
        sss_preprocessed = pd.concat([sss_preprocessed, sss_preprocessed_temp], ignore_index=True)
    
    # aggregate by year and month
    sss_preprocessed_aggregated = sss_preprocessed.groupby(['lat', 'lon', 'year', 'month']).aggregate('mean').reset_index()
    
    # create geodataframe
    sss_preprocessed_aggregated_geo = gpd.GeoDataFrame(sss_preprocessed_aggregated, geometry=gpd.points_from_xy(sss_preprocessed_aggregated.lon, sss_preprocessed_aggregated.lat))
    sss_preprocessed_aggregated_geo.crs = CRS.from_epsg(4326)    
    sss_preprocessed_aggregated_geo = sss_preprocessed_aggregated_geo.drop(['lat', 'lon'], axis=1)
    
    # save geodataframe
    sss_preprocessed_aggregated_geo.to_file(os.path.join(path, f'monthly_sss_{year}.shp'))

Processing 2010...
Processing 2011...
Processing 2012...
Processing 2013...
Processing 2014...
Processing 2015...
Processing 2016...
Processing 2017...
Processing 2018...
CPU times: user 3min 32s, sys: 12.4 s, total: 3min 45s
Wall time: 3min 46s


Check out the last file processed to make sure the output is as desired.

In [12]:
sss_preprocessed_aggregated_geo.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [13]:
sss_preprocessed_aggregated_geo.shape

(165696, 4)

In [14]:
sss_preprocessed_aggregated_geo.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 165696 entries, 0 to 165695
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype   
---  ------    --------------   -----   
 0   year      165696 non-null  int64   
 1   month     165696 non-null  int64   
 2   sss       165696 non-null  float32 
 3   geometry  165696 non-null  geometry
dtypes: float32(1), geometry(1), int64(2)
memory usage: 4.4 MB


In [15]:
sss_preprocessed_aggregated_geo.head()

,year,month,sss,geometry
0,2018,1,35.536095,POINT (60.04323 0.09808)
1,2018,2,35.466061,POINT (60.04323 0.09808)
2,2018,3,35.574512,POINT (60.04323 0.09808)
3,2018,4,35.445744,POINT (60.04323 0.09808)
4,2018,5,35.218952,POINT (60.04323 0.09808)


In [16]:
# free memory
del [sss_preprocessed_temp, sss_preprocessed, sss_preprocessed_aggregated, sss_preprocessed_aggregated_geo]
gc.collect()

0

## Chlorophyll-a concentration

First, a sample is explored before preprocessing all the data and saving it.

In [17]:
path = '../../data/chlorophyll_a_concentration'

In [18]:
file = os.path.join(path, str(years[0]), sorted(os.listdir(os.path.join(path, str(years[0]))))[0])

In [19]:
data = xr.open_dataset(file)

In [20]:
data

<xarray.Dataset>
Dimensions:             (time: 1, lat: 4320, lon: 8640)
Coordinates:
  * lat                 (lat) float64 89.98 89.94 89.9 ... -89.9 -89.94 -89.98
  * lon                 (lon) float64 -180.0 -179.9 -179.9 ... 179.9 179.9 180.0
  * time                (time) datetime64[ns] 2010-01-01
Data variables:
    MERIS_nobs_sum      (time, lat, lon) float32 ...
    MODISA_nobs_sum     (time, lat, lon) float32 ...
    OLCI_nobs_sum       (time, lat, lon) float32 ...
    SeaWiFS_nobs_sum    (time, lat, lon) float32 ...
    VIIRS_nobs_sum      (time, lat, lon) float32 ...
    chlor_a             (time, lat, lon) float32 ...
    chlor_a_log10_bias  (time, lat, lon) float32 ...
    chlor_a_log10_rmsd  (time, lat, lon) float32 ...
    crs                 (time) int32 ...
    total_nobs_sum      (time, lat, lon) float32 ...
Attributes: (12/50)
    Conventions:                       CF-1.7
    Metadata_Conventions:              Unidata Dataset Discovery v1.0
    cdm_data_type:                     Grid
    comment:                           See summary attribute
    creator_email:                     help@esa-oceancolour-cci.org
    creator_name:                      Plymouth Marine Laboratory
    ...                                ...
    stop_date:                         31-JAN-2010 23:59:00.000000
    time_coverage_start:               201001010000Z
    time_coverage_end:                 201001312359Z
    history:                           Source data were: ESACCI-OC-L3S-OC_PRO...
    netcdf_file_type:                  NETCDF4_CLASSIC
    id:                                ESACCI-OC-L3S-CHLOR_A-MERGED-1M_MONTHL...

In [21]:
data.info()

xarray.Dataset {
dimensions:
	time = 1 ;
	lat = 4320 ;
	lon = 8640 ;

variables:
	float32 MERIS_nobs_sum(time, lat, lon) ;
		MERIS_nobs_sum:long_name = Count of the number of observations from the MERIS sensor contributing to this bin cell ;
		MERIS_nobs_sum:number_of_files_composited = 31 ;
	float32 MODISA_nobs_sum(time, lat, lon) ;
		MODISA_nobs_sum:long_name = Count of the number of observations from the MODIS (Aqua) sensor contributing to this bin cell ;
		MODISA_nobs_sum:number_of_files_composited = 31 ;
	float32 OLCI_nobs_sum(time, lat, lon) ;
		OLCI_nobs_sum:long_name = Count of the number of observations from the OLCI sensor contributing to this bin cell ;
		OLCI_nobs_sum:number_of_files_composited = 31 ;
	float32 SeaWiFS_nobs_sum(time, lat, lon) ;
		SeaWiFS_nobs_sum:long_name = Count of the number of observations from the SeaWiFS (GAC and LAC) sensor contributing to this bin cell ;
		SeaWiFS_nobs_sum:number_of_files_composited = 31 ;
	float32 VIIRS_nobs_sum(time, lat, lon) ;
	

In [22]:
# create masks to reduce the data to the desired longitudes and latitudes
mask_lon = (data['lon'] >= lon_min) & (data['lon'] <= lon_max)
mask_lat = (data['lat'] >= lat_min) & (data['lat'] <= lat_max)

In [23]:
%%time

# loop over the desired years, load each file, preprocess it, append it to a geodataframe and save it
for year in years:
    
    print(f'Processing {year}...')
    
    files = os.listdir(os.path.join(path, str(year)))
    chlora_preprocessed = pd.DataFrame(columns=['lat', 'lon', 'chlora', 'year', 'month'])
    
    # load files and apply preprocessing function
    for file in files:
        chlora_preprocessed_temp = preprocess_ecv(os.path.join(path, str(year), file), 'chlor_a', mask_lon, mask_lat)
        chlora_preprocessed_temp = chlora_preprocessed_temp.rename(columns={'chlor_a': 'chlora'})
        chlora_preprocessed = pd.concat([chlora_preprocessed, chlora_preprocessed_temp], ignore_index=True)
    
    # aggregate by year and month
    chlora_preprocessed_aggregated = chlora_preprocessed.groupby(['lat', 'lon', 'year', 'month']).aggregate('mean').reset_index()
    
    # compute log
    chlora_preprocessed_aggregated['chlora'] = np.log(chlora_preprocessed_aggregated['chlora'])
    
    # create geodataframe
    chlora_preprocessed_aggregated_geo = gpd.GeoDataFrame(chlora_preprocessed_aggregated, geometry=gpd.points_from_xy(chlora_preprocessed_aggregated.lon, chlora_preprocessed_aggregated.lat))
    chlora_preprocessed_aggregated_geo.crs = CRS.from_epsg(4326)
    chlora_preprocessed_aggregated_geo = chlora_preprocessed_aggregated_geo.drop(['lat', 'lon'], axis=1)
    
    # save geodataframe
    chlora_preprocessed_aggregated_geo.to_file(os.path.join(path, f'monthly_chlora_{year}.shp'))

Processing 2010...
Processing 2011...
Processing 2012...
Processing 2013...
Processing 2014...
Processing 2015...
Processing 2016...
Processing 2017...
Processing 2018...
CPU times: user 1h 31min 26s, sys: 5min 52s, total: 1h 37min 19s
Wall time: 1h 37min 24s


Check out the last file processed to make sure the output is as desired.

In [24]:
chlora_preprocessed_aggregated_geo.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [25]:
chlora_preprocessed_aggregated_geo.shape

(4782730, 4)

In [26]:
chlora_preprocessed_aggregated_geo.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 4782730 entries, 0 to 4782729
Data columns (total 4 columns):
 #   Column    Dtype   
---  ------    -----   
 0   year      int64   
 1   month     int64   
 2   chlora    float32 
 3   geometry  geometry
dtypes: float32(1), geometry(1), int64(2)
memory usage: 127.7 MB


In [27]:
chlora_preprocessed_aggregated_geo.head()

,year,month,chlora,geometry
0,2018,1,-1.571544,POINT (60.02083 0.02083)
1,2018,2,-1.876267,POINT (60.02083 0.02083)
2,2018,3,-1.264938,POINT (60.02083 0.02083)
3,2018,4,-2.081722,POINT (60.02083 0.02083)
4,2018,5,-2.214021,POINT (60.02083 0.02083)


In [28]:
# free memory
del [chlora_preprocessed_temp, chlora_preprocessed, chlora_preprocessed_aggregated, chlora_preprocessed_aggregated_geo]
gc.collect()

0

## Land surface temperature

First, a sample is explored before preprocessing all the data and saving it.

In [29]:
path = '../../data/land_surface_temperature'

In [30]:
file = os.path.join(path, str(years[0]), sorted(os.listdir(os.path.join(path, str(years[0]))))[0])

In [31]:
data = xr.open_dataset(file)

In [32]:
data

<xarray.Dataset>
Dimensions:  (time: 1, lon: 801, lat: 801)
Coordinates:
  * time     (time) datetime64[ns] 2010-01-01
  * lon      (lon) float64 60.0 60.05 60.1 60.15 60.2 ... 99.85 99.9 99.95 100.0
  * lat      (lat) float64 0.0 0.05 0.1 0.15 0.2 ... 39.8 39.85 39.9 39.95 40.0
Data variables:
    lst      (time, lat, lon) float32 ...
Attributes: (12/43)
    CDI:                        Climate Data Interface version 1.9.9 (https:/...
    Conventions:                CF-1.8
    source:                     ESA LST CCI ATSR_3 L3U V3.00
    institution:                University of Leicester
    title:                      ESA LST CCI land surface temperature time ser...
    history:                    Mon Apr 11 18:42:27 2022: cdo -L -z zip_9 cop...
    ...                         ...
    geospatial_lat_resolution:  0.01
    key_variables:              land_surface_temperature
    format_version:             CCI Data Standards v2.2
    spatial_resolution:         0.01 degree
    doi:                        10.5285/ef8ce37b6af24469a2a4bdc31d3db27d
    CDO:                        Climate Data Operators version 1.9.9 (https:/...

In [33]:
data.info()

xarray.Dataset {
dimensions:
	time = 1 ;
	lon = 801 ;
	lat = 801 ;

variables:
	datetime64[ns] time(time) ;
		time:standard_name = time ;
		time:long_name = reference time of file ;
		time:axis = T ;
	float64 lon(lon) ;
		lon:standard_name = longitude ;
		lon:long_name = Longitude ;
		lon:units = degrees_east ;
		lon:axis = X ;
	float64 lat(lat) ;
		lat:standard_name = latitude ;
		lat:long_name = Latitude ;
		lat:units = degrees_north ;
		lat:axis = Y ;
	float32 lst(time, lat, lon) ;
		lst:long_name = land surface temperature ;
		lst:units = kelvin ;

// global attributes:
	:CDI = Climate Data Interface version 1.9.9 (https://mpimet.mpg.de/cdi) ;
	:Conventions = CF-1.8 ;
	:source = ESA LST CCI ATSR_3 L3U V3.00 ;
	:institution = University of Leicester ;
	:title = ESA LST CCI land surface temperature time series from multiple IR sensors on LEO platforms. ;
	:history = Mon Apr 11 18:42:27 2022: cdo -L -z zip_9 copy /tmp/nctoolkithwzuudbsnctoolkittmp7zllej2b.nc ESACCI-LST-L3S-LST-IRCDR_-

In [34]:
%%time

# loop over the desired years, load files, preprocess them, create a geodataframe and save it
for year in years:
    
    print(f'Processing {year}...')
    
    # load data
    lst_raw = xr.open_mfdataset(os.path.join(path, str(year)) + '/*.nc')
    
    # drop na
    lst_preprocessed = lst_raw.to_dataframe().dropna(subset=['lst']).reset_index()
    
    # get year and month
    lst_preprocessed['year'] = lst_preprocessed['time'].apply(lambda x: x.year)
    lst_preprocessed['month'] = lst_preprocessed['time'].apply(lambda x: x.month)
    
    # drop time
    lst_preprocessed = lst_preprocessed.drop('time', axis=1)
    
    # aggregate by year and month
    lst_preprocessed_aggregated = lst_preprocessed.groupby(['lat', 'lon', 'year', 'month']).aggregate('mean').reset_index()
    
    # create geodataframe
    lst_preprocessed_aggregated_geo = gpd.GeoDataFrame(lst_preprocessed_aggregated, geometry=gpd.points_from_xy(lst_preprocessed_aggregated.lon, lst_preprocessed_aggregated.lat))
    lst_preprocessed_aggregated_geo.crs = CRS.from_epsg(4326)
    lst_preprocessed_aggregated_geo = lst_preprocessed_aggregated_geo.drop(['lat', 'lon'], axis=1)
    
    # save geodataframe
    lst_preprocessed_aggregated_geo.to_file(os.path.join(path, f'monthly_lst_{year}.shp'))

Processing 2010...
Processing 2011...
Processing 2012...
Processing 2013...
Processing 2014...
Processing 2015...
Processing 2016...
Processing 2017...
Processing 2018...
CPU times: user 1h 20min 15s, sys: 5min 20s, total: 1h 25min 36s
Wall time: 1h 25min 22s


Check out the last file processed to make sure the output is as desired.

In [35]:
lst_preprocessed_aggregated_geo.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [36]:
lst_preprocessed_aggregated_geo.shape

(3226495, 4)

In [37]:
lst_preprocessed_aggregated_geo.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 3226495 entries, 0 to 3226494
Data columns (total 4 columns):
 #   Column    Dtype   
---  ------    -----   
 0   year      int64   
 1   month     int64   
 2   lst       float32 
 3   geometry  geometry
dtypes: float32(1), geometry(1), int64(2)
memory usage: 86.2 MB


In [38]:
lst_preprocessed_aggregated_geo.head()

,year,month,lst,geometry
0,2018,4,308.109985,POINT (99.65000 0.05000)
1,2018,6,306.709991,POINT (99.65000 0.05000)
2,2018,7,307.760010,POINT (99.65000 0.05000)
3,2018,8,307.919983,POINT (99.65000 0.05000)
4,2018,9,307.039978,POINT (99.65000 0.05000)


In [39]:
# free memory
del [lst_raw, lst_preprocessed, lst_preprocessed_aggregated, lst_preprocessed_aggregated_geo]
gc.collect()

1280